Recover stale shards in Elasticsearch cluster

In [ ]:
//Import packages

#r "nuget:NEST,6.8.11"


Create Elastics client

In [ ]:
Uri host = new Uri("");
string username = "";
string password = "";

In [ ]:
using Nest;
using System.Net.Security;
using System.Security.Cryptography.X509Certificates;

static ElasticClient GetElasticClient(Uri host, string username, string password)
{
    var connection = new ConnectionSettings(host);
    
    connection.BasicAuthentication(username, password);
    connection.MaximumRetries(3);
    connection.RequestTimeout(TimeSpan.FromMinutes(3.0));
    connection.MaxRetryTimeout(TimeSpan.FromMinutes(3.0));
    connection.ServerCertificateValidationCallback((object sender, X509Certificate cert, X509Chain chain, SslPolicyErrors errors) => true);
    connection.DisableDirectStreaming();
    
    return new ElasticClient(connection);
}

var elasticClient = GetElasticClient(host, username, password);

Get nodes

In [ ]:
static async Task<IEnumerable<CatNodesRecord>> GetNodes(ElasticClient elasticClient)
{
    var nodesResponse = await elasticClient.CatNodesAsync();

    if (!nodesResponse.IsValid)
    {
        Console.WriteLine("Invalid nodes response");

        return Enumerable.Empty<CatNodesRecord>();
    }

    return nodesResponse.Records;
}

var nodes = await GetNodes(elasticClient);

if (!nodes.Any())
{
    Console.WriteLine("No nodes");
}

Get shards

In [ ]:
static async Task<IEnumerable<CatShardsRecord>> GetShards(ElasticClient elasticClient)
{
    var shardsResponse = await elasticClient.CatShardsAsync();

    if (!shardsResponse.IsValid)
    {
        Console.WriteLine("Invalid shards response");

        return Enumerable.Empty<CatShardsRecord>();
    }

    var shards = shardsResponse.Records
        .Where(t => t.State == "UNASSIGNED" && t.PrimaryOrReplica == "p")
        .ToList();

    return shards;
}

var shards = await GetShards(elasticClient);

if (!shards.Any())
{
    Console.WriteLine("No UNASSIGNED shards");
}

Fix shards

In [ ]:
static async Task FixShards(ElasticClient elasticClient, IEnumerable<CatShardsRecord> shards, IEnumerable<CatNodesRecord> nodes)
{
    foreach (var shard in shards)
    {
        var shardNumber = int.Parse(shard.Shard);

        var isFixed = false;

        foreach (var node in nodes)
        {
            var rerouteResponse = await elasticClient.ClusterRerouteAsync(rerouteDescriptor => rerouteDescriptor
                .AllocateStalePrimary(staleDescriptor => staleDescriptor
                    .Index(shard.Index)
                    .Shard(shardNumber)
                    .Node(node.Name)
                    .AcceptDataLoss(true)));

            if (rerouteResponse.IsValid)
            {
                isFixed = true;

                break;
            }
        }

        if (isFixed)
        {
            Console.WriteLine($"Shard:{shardNumber};Index:{shard.Index}; fixed");
        }
        else
        {
            Console.WriteLine($"Shard:{shardNumber};Index:{shard.Index}; NOT FIXED");
        }
    }
}

await FixShards(elasticClient, shards, nodes);

Check for any stale shards left

In [ ]:
shards = await GetShards(elasticClient);

Console.WriteLine();

if (!shards.Any())
{
    Console.WriteLine("All UNASSIGNED shards fixed");
}
else
{
    Console.WriteLine($"{shards.Count()} UNASSIGNED shards not fixed");
}